In [1]:
%run 'Setup.py'

8 different classes: Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock.
objective 1: construct a classifier which, based on the features of a song, predicts its genre
objective 2: estimate its generalisation error under the 0–1 loss.
Features are real-valued, correspond to summary statistics (mean, sd, skewness, kurtosis, median, min, max) of 
time series of various music features, such as the chromagram or the Mel-frequency cepstrum.
Feature description: 

Feature description: 
chroma_cens: Chroma Energy Normalized (CENS, 12 chroma) - 84 features
chroma_cqt: Constant-Q chromagram (12 chroma) - 84 features
chroma_stft: Chromagram (12 chroma) - 84 features
mfcc: Mel-frequency cepstrum (20 coefficients) - 140 features
rmse: Root-mean-square - 7 features
spectral_bandwidth: Spectral bandwidth - 7 features
spectral_centroid: Spectral centroid - 7 features
spectral_contrast: Spectral contrast (7 frequency bands) - 49 features
spectral_rolloff: Roll-off freque

In [2]:
# Prepare data
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_np.ravel()) #

# Split training data into training and temporary validation sets
X_train, X_temp, Y_train, Y_temp = train_test_split(x_train, y_train_encoded, test_size=0.4, random_state=42)

# Split the temporary validation set into validation and test sets
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)  
X_real_test_scaled = scaler.transform(x_test_np)

/Users/max/anaconda3/envs/sml-practical/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# KNN Bagging

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

# Number of base estimators
n_estimators = 10  # You can experiment with this number

# Create a BaggingClassifier with KNN as base estimators
bagging_clf = BaggingClassifier(
    estimator=KNeighborsClassifier(n_neighbors=1), 
    n_estimators=n_estimators, 
    max_samples=1.0 / n_estimators, 
    bootstrap=True, 
    random_state=42,
    n_jobs=-1  # Use all cores
)

# Fit the BaggingClassifier
bagging_clf.fit(X_train_scaled, Y_train)

# Evaluate the model
train_accuracy = bagging_clf.score(X_train_scaled, Y_train)
val_accuracy = bagging_clf.score(X_val_scaled, Y_val)
test_accuracy = bagging_clf.score(X_test_scaled, Y_test)

print(f"Training accuracy: {train_accuracy}")
print(f"Validation accuracy: {val_accuracy}")
print(f"Test accuracy: {test_accuracy}")

# Predict on real test set if needed
real_test_predictions = bagging_clf.predict(X_real_test_scaled)


Training accuracy: 0.5222222222222223
Validation accuracy: 0.4275
Test accuracy: 0.39916666666666667


# KNN  Ensembling

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import mode

# Define a range of k values
k_values = [1, 8, 64]

# Train a KNN model for each k
knn_models = [KNeighborsClassifier(n_neighbors=k).fit(X_train_scaled, Y_train) for k in k_values]

# Predict on the validation set with each model
val_predictions = [model.predict(X_val_scaled) for model in knn_models]

# Combine predictions: Take the mode of predictions across the different models
combined_val_predictions = mode(val_predictions, axis=0)[0][0]

# Evaluate accuracy
val_accuracy = np.mean(combined_val_predictions == Y_val)
print(f"Validation accuracy with combined KNN models: {val_accuracy}")


Validation accuracy with combined KNN models: 0.12666666666666668


## KNN on feature subsets

In [3]:
print(x_train.columns.tolist())

['chroma_cens_kurtosis_01', 'chroma_cens_kurtosis_02', 'chroma_cens_kurtosis_03', 'chroma_cens_kurtosis_04', 'chroma_cens_kurtosis_05', 'chroma_cens_kurtosis_06', 'chroma_cens_kurtosis_07', 'chroma_cens_kurtosis_08', 'chroma_cens_kurtosis_09', 'chroma_cens_kurtosis_10', 'chroma_cens_kurtosis_11', 'chroma_cens_kurtosis_12', 'chroma_cens_max_01', 'chroma_cens_max_02', 'chroma_cens_max_03', 'chroma_cens_max_04', 'chroma_cens_max_05', 'chroma_cens_max_06', 'chroma_cens_max_07', 'chroma_cens_max_08', 'chroma_cens_max_09', 'chroma_cens_max_10', 'chroma_cens_max_11', 'chroma_cens_max_12', 'chroma_cens_mean_01', 'chroma_cens_mean_02', 'chroma_cens_mean_03', 'chroma_cens_mean_04', 'chroma_cens_mean_05', 'chroma_cens_mean_06', 'chroma_cens_mean_07', 'chroma_cens_mean_08', 'chroma_cens_mean_09', 'chroma_cens_mean_10', 'chroma_cens_mean_11', 'chroma_cens_mean_12', 'chroma_cens_median_01', 'chroma_cens_median_02', 'chroma_cens_median_03', 'chroma_cens_median_04', 'chroma_cens_median_05', 'chroma_ce

## Function to extract feature subsets

In [4]:
def split_features_by_type(X, feature_structure):
    """
    Splits the dataset into subsets based on the feature structure provided.

    :param X: numpy array, the dataset to be split (features only)
    :param feature_structure: dict, keys are feature names and values are the number of features of that type
    :return: dict of feature subsets
    """
    feature_subsets = {}
    start_idx = 0
    
    for feature_name, feature_count in feature_structure.items():
        end_idx = start_idx + feature_count
        feature_subsets[feature_name] = X[:, start_idx:end_idx]
        start_idx = end_idx
    
    return feature_subsets

# Define the structure of your features based on the information you've provided
feature_structure = {
    'chroma_cens': 84,
    'chroma_cqt': 84,
    'chroma_stft': 84,
    'mfcc': 140,
    'rmse': 7,
    'spectral_bandwidth': 7,
    'spectral_centroid': 7,
    'spectral_contrast': 49,
    'spectral_rolloff': 7,
    'tonnetz': 42,
    'zcr': 7
}

# Example usage with a hypothetical dataset X_train_scaled
# This would be your preprocessed and scaled training data as a NumPy array
feature_subsets = split_features_by_type(X_train_scaled, feature_structure)

# Now feature_subsets is a dictionary where, for example,
# feature_subsets['mfcc'] contains only the MFCC features of the dataset.


In [5]:
from sklearn.neighbors import KNeighborsClassifier

# Dictionary to store the trained KNN models for each feature subset
knn_models = {}

# Train a KNN model for each feature subset
from sklearn.model_selection import cross_val_score

# Dictionary to store the best KNN models for each feature subset
best_knn_models = {}

# Train a KNN model for each feature subset and find the best k using cross-validation
for feature_name, X_subset in feature_subsets.items():
    best_score = 0
    best_k = 1
    # Try different values of k
    for k in range(1, 16):  # Let's try k from 1 to 15 as an example
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X_subset, Y_train, cv=5)
        mean_score = scores.mean()
        if mean_score > best_score:
            best_score = mean_score
            best_k = k

    # Train a new KNN model on the full training set with the best k
    best_knn = KNeighborsClassifier(n_neighbors=best_k)
    best_knn.fit(X_subset, Y_train)
    best_knn_models[feature_name] = best_knn
    print(f"Best K for {feature_name} features: {best_k} with cross-validation score: {best_score}")

# Now best_knn_models dictionary contains the best KNN model for each feature subset


# Now knn_models dictionary contains a trained KNN model for each feature subset
# For example, knn_models['mfcc'] is the KNN model trained on the MFCC features

# To make predictions, use the corresponding model for each feature subset
# For instance, for MFCC features:
# predictions_mfcc = knn_models['mfcc'].predict(feature_subsets['mfcc'])

from scipy.stats import mode

# Assume we have a validation set X_val_scaled
# Split it using the same function we defined earlier
val_feature_subsets = split_features_by_type(X_val_scaled, feature_structure)

# Gather predictions from all models on the validation set
val_predictions = []
for feature_name, model in best_knn_models.items():
    # Ensure that we predict on the correct feature subset
    X_val_subset = val_feature_subsets[feature_name]
    predictions = model.predict(X_val_subset)
    val_predictions.append(predictions)

# Combine predictions using majority voting
combined_val_predictions = mode(val_predictions, axis=0).mode

# Calculate accuracy or any other metric based on the combined predictions
val_accuracy = np.mean(combined_val_predictions.ravel() == Y_val)
print(f"Validation accuracy with combined KNN models: {val_accuracy}")



Best K for chroma_cens features: 9 with cross-validation score: 0.2663888888888889
Best K for chroma_cqt features: 13 with cross-validation score: 0.27361111111111114



KeyboardInterrupt



## Weighted Voting

In [38]:
# Calculate the validation accuracy for each feature subset and use it as weight for voting
weights = []
for feature_name, model in best_knn_models.items():
    X_val_subset = val_feature_subsets[feature_name]
    val_accuracy = model.score(X_val_subset, Y_val)
    weights.append(val_accuracy)
    print(f"Validation accuracy for {feature_name} features: {val_accuracy}")

# Normalize weights so they sum up to 1
weights = np.array(weights) / np.sum(weights)

# Split the test set using the same feature structure
test_feature_subsets = split_features_by_type(X_test_scaled, feature_structure)

# Predict on the test set with each model and weight the predictions
weighted_test_predictions = np.zeros((X_test_scaled.shape[0], len(np.unique(Y_train))), dtype=float)

for i, (feature_name, model) in enumerate(best_knn_models.items()):
    X_test_subset = test_feature_subsets[feature_name]
    predictions = model.predict_proba(X_test_subset)
    weighted_predictions = predictions * weights[i]
    weighted_test_predictions += weighted_predictions

# Combine weighted predictions by taking the argmax to get final predictions
combined_test_predictions = np.argmax(weighted_test_predictions, axis=1)

# Calculate accuracy based on the combined weighted predictions
test_accuracy = np.mean(combined_test_predictions == Y_test)
print(f"Test accuracy with combined KNN models using weighted voting: {test_accuracy}")


Validation accuracy for chroma_cens features: 0.27
Validation accuracy for chroma_cqt features: 0.2658333333333333
Validation accuracy for chroma_stft features: 0.325
Validation accuracy for mfcc features: 0.4741666666666667
Validation accuracy for rmse features: 0.22916666666666666
Validation accuracy for spectral_bandwidth features: 0.29583333333333334
Validation accuracy for spectral_centroid features: 0.3416666666666667
Validation accuracy for spectral_contrast features: 0.41083333333333333
Validation accuracy for spectral_rolloff features: 0.3225
Validation accuracy for tonnetz features: 0.25
Validation accuracy for zcr features: 0.295
Test accuracy with combined KNN models using weighted voting: 0.5208333333333334


In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Define a list of distance metrics to try
distance_metrics = ['euclidean', 'manhattan', 'minkowski']

# Define a range of k values to try
k_values = range(1, 16)  # Example: trying k from 1 to 15

# Dictionary to store the best KNN model for each feature subset
best_knn_models_dm_k = {}

# Dictionary to store the best combination of distance metric and k for each feature subset
best_combinations = {}

for feature_name, X_subset in feature_subsets.items():
    best_score = 0
    best_combination = {'metric': '', 'k': 0}
    # Iterate over each distance metric
    for metric in distance_metrics:
        # Iterate over each value of k
        for k in k_values:
            # Create a KNN model with the current metric and k
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            # Perform cross-validation and compute the mean score
            scores = cross_val_score(knn, X_subset, Y_train, cv=5)
            mean_score = scores.mean()
            # Update the best combination if the current model performs better
            if mean_score > best_score:
                best_score = mean_score
                best_combination['metric'] = metric
                best_combination['k'] = k
    
    # Once the best combination is found, retrain the model on the full training set
    best_knn = KNeighborsClassifier(n_neighbors=best_combination['k'], metric=best_combination['metric'])
    best_knn.fit(X_subset, Y_train)
    best_knn_models_dm_k[feature_name] = best_knn
    best_combinations[feature_name] = best_combination
    print(f"Best combination for {feature_name} features: k={best_combination['k']}, metric={best_combination['metric']} with cross-validation score: {best_score:.2f}")


Best combination for chroma_cens features: k=9, metric=euclidean with cross-validation score: 0.27
Best combination for chroma_cqt features: k=13, metric=euclidean with cross-validation score: 0.27
Best combination for chroma_stft features: k=13, metric=euclidean with cross-validation score: 0.30
Best combination for mfcc features: k=15, metric=manhattan with cross-validation score: 0.48
Best combination for rmse features: k=14, metric=manhattan with cross-validation score: 0.24
Best combination for spectral_bandwidth features: k=15, metric=manhattan with cross-validation score: 0.29
Best combination for spectral_centroid features: k=14, metric=manhattan with cross-validation score: 0.33
Best combination for spectral_contrast features: k=13, metric=manhattan with cross-validation score: 0.42
Best combination for spectral_rolloff features: k=14, metric=manhattan with cross-validation score: 0.31
Best combination for tonnetz features: k=14, metric=manhattan with cross-validation score: 0

In [42]:
# Calculate the validation accuracy for each feature subset and use it as weight for voting
weights = []
for feature_name, model in best_knn_models_dm_k.items():
    X_val_subset = val_feature_subsets[feature_name]
    val_accuracy = model.score(X_val_subset, Y_val)
    weights.append(val_accuracy)
    print(f"Validation accuracy for {feature_name} features: {val_accuracy:.2f}")

# Normalize weights so they sum up to 1
weights = np.array(weights) / np.sum(weights)

# Split the test set using the same feature structure
test_feature_subsets = split_features_by_type(X_test_scaled, feature_structure)

# Predict on the test set with each model and weight the predictions
weighted_test_predictions = np.zeros((X_test_scaled.shape[0], 8), dtype=float)

for i, (feature_name, model) in enumerate(best_knn_models_dm_k.items()):
    X_test_subset = test_feature_subsets[feature_name]
    predictions = model.predict_proba(X_test_subset)
    weighted_predictions = predictions * weights[i]
    weighted_test_predictions += weighted_predictions

# Combine weighted predictions by taking the argmax to get final predictions
combined_test_predictions = np.argmax(weighted_test_predictions, axis=1)

# Calculate accuracy based on the combined weighted predictions
test_accuracy = np.mean(combined_test_predictions == Y_test)
print(f"Test accuracy with combined KNN models using weighted voting: {test_accuracy}")


Validation accuracy for chroma_cens features: 0.27
Validation accuracy for chroma_cqt features: 0.27
Validation accuracy for chroma_stft features: 0.33
Validation accuracy for mfcc features: 0.49
Validation accuracy for rmse features: 0.23
Validation accuracy for spectral_bandwidth features: 0.31
Validation accuracy for spectral_centroid features: 0.35
Validation accuracy for spectral_contrast features: 0.42
Validation accuracy for spectral_rolloff features: 0.33
Validation accuracy for tonnetz features: 0.28
Validation accuracy for zcr features: 0.31
Test accuracy with combined KNN models using weighted voting: 0.5116666666666667


In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Define a list of distance metrics to try
distance_metrics = ['euclidean', 'manhattan', 'minkowski']

# Define a range of k values to try
k_values = range(1, 16)  # Example: trying k from 1 to 15

# Dictionary to store the best KNN model for each feature subset
best_knn_models_dm_k = {}

# Dictionary to store the best combination of distance metric and k for each feature subset
best_combinations = {}
cv_scores = []

for feature_name, X_subset in feature_subsets.items():
    best_score = 0
    best_combination = {'metric': '', 'k': 0}
    # Iterate over each distance metric
    for metric in distance_metrics:
        # Iterate over each value of k
        for k in k_values:
            # Create a KNN model with the current metric and k
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            # Perform cross-validation and compute the mean score
            scores = cross_val_score(knn, X_subset, Y_train, cv=5)
            mean_score = scores.mean()
            # Update the best combination if the current model performs better
            if mean_score > best_score:
                best_score = mean_score
                best_combination['metric'] = metric
                best_combination['k'] = k
        
    # Once the best combination is found, retrain the model on the full training set
    best_knn = KNeighborsClassifier(n_neighbors=best_combination['k'], metric=best_combination['metric'])
    best_knn.fit(X_subset, Y_train)
    best_knn_models_dm_k[feature_name] = best_knn
    best_combinations[feature_name] = best_combination
    print(f"Best combination for {feature_name} features: k={best_combination['k']}, metric={best_combination['metric']} with cross-validation score: {best_score:.2f}")
    cv_scores.append(best_score)
    
weights = cv_scores / np.sum(cv_scores)

Best combination for chroma_cens features: k=9, metric=euclidean with cross-validation score: 0.27
Best combination for chroma_cqt features: k=13, metric=euclidean with cross-validation score: 0.27
Best combination for chroma_stft features: k=13, metric=euclidean with cross-validation score: 0.30
Best combination for mfcc features: k=15, metric=manhattan with cross-validation score: 0.48
Best combination for rmse features: k=14, metric=manhattan with cross-validation score: 0.24
Best combination for spectral_bandwidth features: k=15, metric=manhattan with cross-validation score: 0.29
Best combination for spectral_centroid features: k=14, metric=manhattan with cross-validation score: 0.33
Best combination for spectral_contrast features: k=13, metric=manhattan with cross-validation score: 0.42
Best combination for spectral_rolloff features: k=14, metric=manhattan with cross-validation score: 0.31
Best combination for tonnetz features: k=14, metric=manhattan with cross-validation score: 0

## Use Logistic Reg as Meta-Learner 

In [46]:
# Generate meta-features for the validation set
val_meta_features = np.column_stack([
    model.predict_proba(val_feature_subsets[feature_name]) for feature_name, model in best_knn_models_dm_k.items()
])

from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(val_meta_features, Y_val)  # Assuming Y_val is the validation set labels

# Generate meta-features for the test set
test_meta_features = np.column_stack([
    model.predict_proba(test_feature_subsets[feature_name]) for feature_name, model in best_knn_models_dm_k.items()
])

# Final predictions using the meta-learner
final_predictions = meta_model.predict(test_meta_features)

# Calculate accuracy based on the combined weighted predictions
test_accuracy = np.mean(final_predictions == Y_test)
print(f"Test accuracy with stacked KNN models: {test_accuracy}")



Test accuracy with stacked KNN models: 0.5533333333333333


## Use simple NN as meta-learner

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Check if MPS (Apple Silicon GPU) is available, otherwise use CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: mps


In [48]:
# Convert meta-features and labels to PyTorch tensors
X_val_tensor = torch.tensor(val_meta_features).float().to(device)
Y_val_tensor = torch.tensor(Y_val).long().to(device)  # Assuming Y_val is already encoded as integers

X_test_tensor = torch.tensor(test_meta_features).float().to(device)

# Create DataLoader for the validation set
batch_size = 32
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


In [49]:
class MetaLearnerNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MetaLearnerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
input_size = val_meta_features.shape[1]  # Number of meta-features
num_classes = 8  # Assuming 8 output classes
model = MetaLearnerNN(input_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [50]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in val_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 2.0166
Epoch [2/100], Loss: 1.9187
Epoch [3/100], Loss: 1.2562
Epoch [4/100], Loss: 1.5095
Epoch [5/100], Loss: 1.6083
Epoch [6/100], Loss: 1.4226
Epoch [7/100], Loss: 1.3978
Epoch [8/100], Loss: 0.8296
Epoch [9/100], Loss: 1.1557
Epoch [10/100], Loss: 1.5398
Epoch [11/100], Loss: 1.2438
Epoch [12/100], Loss: 0.9344
Epoch [13/100], Loss: 0.9194
Epoch [14/100], Loss: 1.4493
Epoch [15/100], Loss: 0.7873
Epoch [16/100], Loss: 0.8417
Epoch [17/100], Loss: 1.3168
Epoch [18/100], Loss: 1.0663
Epoch [19/100], Loss: 1.1252
Epoch [20/100], Loss: 1.0421
Epoch [21/100], Loss: 0.9747
Epoch [22/100], Loss: 0.9928
Epoch [23/100], Loss: 1.2502
Epoch [24/100], Loss: 1.7411
Epoch [25/100], Loss: 1.5389
Epoch [26/100], Loss: 1.1095
Epoch [27/100], Loss: 1.0328
Epoch [28/100], Loss: 1.2746
Epoch [29/100], Loss: 1.2590
Epoch [30/100], Loss: 1.7449
Epoch [31/100], Loss: 1.4318
Epoch [32/100], Loss: 0.9114
Epoch [33/100], Loss: 1.1084
Epoch [34/100], Loss: 0.8470
Epoch [35/100], Loss: 0

In [51]:
# Predict on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)

# Convert predictions back to numpy for accuracy calculation
predicted_np = predicted.cpu().numpy()
test_accuracy = (predicted_np == Y_test).mean()
print(f"Test accuracy with PyTorch NN stacked model: {test_accuracy}")


Test accuracy with PyTorch NN stacked model: 0.5266666666666666
